In [1]:
from pyhdf.SD  import SD, SDC
from pyhdf.V import *
from pyhdf.HDF import *
import matplotlib.pyplot as plt
import os
os.environ['PROJ_LIB'] = 'D:\\program\\anaconda3\\Library\\share\\'
#os.environ['PROJ_LIB'] = '/home/C.jerryjerry9/anaconda3/share/proj/'
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
import pprint
import gc 

#####Fuction for extracting variable names from hdf file
def HDFvars(File):
    hdfFile = SD(File, SDC.READ)
    dsets = hdfFile.datasets()
    k = []
    for key in dsets.keys():
        k.append(key)
    k.sort()
    hdfFile.end() # close the file
    return k
#####Function end 

In [ ]:
filename = '檔案.hdf'
#讀取vdata
reading_vdata = HDF(filename, HC.READ).vstart()
vdata =reading_vdata.vdatainfo()

pprint.pprint(vdata[:])##把vdata存的變數名稱輸出
members = ['vdata代碼'] ##我們想提出的geolocation data
nrecs = []
names = []
for ref in members:
 #       # Vdata tag
    vd = reading_vdata.attach(ref)
    nrecs.append(vd.inquire()[0])  ## number of records of the Vdata
    names.append(vd.inquire()[-1]) ## name of the Vdata
    vd.detach()

print(names)##輸出的變數名稱
print(nrecs)##變數矩陣的長度

# Read geolocation dataset.
#['UTC_start', 'Latitude', 'Longitude', 'Data_quality', 'MODIS_Cloud_Fraction','Navigation_land_sea_flag']
idx = names.index('UTC_start')
var = reading_vdata.attach(members[idx])
UTC_time = var.read(nrecs[idx])

idx = names.index('Latitude')
var = reading_vdata.attach(members[idx])
latitude = var.read(nrecs[idx])

idx = names.index('Longitude',)
var = reading_vdata.attach(members[idx])
longitude = var.read(nrecs[idx])

idx = names.index('MODIS_Cloud_Fraction',)
var = reading_vdata.attach(members[idx])
MODIS_cloud_fraction = var.read(nrecs[idx])

idx = names.index('Data_quality',)
var = reading_vdata.attach(members[idx])
data_quality = var.read(nrecs[idx])

del reading_vdata
print(gc.collect())

In [ ]:
lat_demo = pd.DataFrame(latitude[:], columns= ['latitude'])
print(lat_demo.shape)
print(lat_demo.head())

In [ ]:
lon_demo = pd.DataFrame(longitude[:], columns= ['longitude'])
print(lon_demo.shape)
print(lon_demo.head())

In [13]:
#以台灣為中心畫圖，所以要調整一下經度的表示方式
mo_longitude=np.asarray(longitude)
shape1 = mo_longitude.shape

for i in range(0,shape1[0]):
    if mo_longitude[i] < -60:
        mo_longitude[i]=mo_longitude[i]+360


In [ ]:
#先把這個hdf檔的軌道畫出來看看
#尋找你感興趣的區域
fig = plt.figure(figsize=(16,12)) 
m = Basemap(llcrnrlon=-60, urcrnrlon=300., llcrnrlat=-90., urcrnrlat=90.,resolution='l')
m.drawcoastlines(linewidth=0.5)
m.drawparallels(np.arange(-90., 120., 30.), labels=[1, 0, 0, 0])
m.drawmeridians(np.arange(-180., 181., 30.), labels=[0, 0, 0, 1])
m.scatter(mo_longitude[:], latitude[:] , c=MODIS_cloud_fraction[:] ,cmap=plt.cm.get_cmap('jet',10), vmin=10, vmax=120)
#plt.show()

In [ ]:
#利用經緯度矩陣的值找出感興趣的區域在矩陣中的位置
mo_longitude=np.asarray(longitude)
mo_latitude=np.asarray(latitude)
shape1 = mo_longitude.shape
lonlat_mask= np.zeros((shape1[0],1),dtype=int)

for i in range(0,shape1[0]):
    if mo_longitude[i] >'longitude_left_edge'  and  mo_longitude[i]<'longitude_right_edge'  and  mo_latitude[i] > 'latitude_bottom_edge'  and  mo_latitude[i] <  'latitude_top_edge':
        lonlat_mask[i]= 1##有在所選經緯度涵蓋範圍的矩陣會被設成'1'
        
region_array_size=sum(lonlat_mask)
print("Array length over the select region:")
print(sum(region_array_size))


In [ ]:
HDFvars(filename)

In [ ]:
# Read dataset.
hdfFile = SD(filename, SDC.READ)##SDS資料輸出
Reflectivity = hdfFile.select('ref_array')##輸出變數
Cloud_mask = hdfFile.select('cloud_mask_array')
Height = hdfFile.select('height_array')
pprint.pprint(Reflectivity.attributes())##輸出變數的基本資料

In [ ]:
demo_longitude=np.asarray(longitude)
demo_latitude=np.asarray(latitude)
demo_Reflectivity=np.asarray(Reflectivity[:,:])
demo_Height=np.asarray(Height[:,:])
print(demo_Reflectivity.shape)
print(demo_Height.shape)
Ref_array_demo = pd.DataFrame(Reflectivity[:,:] ,columns=demo_Height[0,:] ,index=demo_latitude[:,0])
Ref_array_demo.head()

In [20]:
#從完整變數矩陣中取出感興趣的區域的矩陣
region_Reflectivity= np.zeros((region_array_size[0],125))
region_Cloud_mask= np.zeros((region_array_size[0],125))
region_height= np.zeros((region_array_size[0],125))

region_lat= np.zeros((region_array_size[0],1))
region_lon= np.zeros((region_array_size[0],1))
k=0
for i in range(0,shape1[0]):
    if lonlat_mask[i]>0:
        region_Reflectivity[k,:]= Reflectivity[i,:]/100
        region_Cloud_mask[k,:]= Cloud_mask[i,:]
        region_height[k,:]=Height[i,:]
        region_lat[k,0]=mo_latitude[i]
        region_lon[k,0]=mo_longitude[i]
        k=k+1

In [ ]:
Ref_array_demo = pd.DataFrame(region_Reflectivity[:,:] ,columns=region_height[0,:] ,index=region_lat[:,0])
print(Ref_array_demo.shape)
Ref_array_demo.head()

In [24]:
#先轉置矩陣再把columns倒過來，為了畫圖
Ref= region_Reflectivity[:,:].T
r_Ref = Ref[::-1]
c_mask=region_Cloud_mask[:,:].T
r_c_mask = c_mask[::-1]
Hef= region_height[:,:].T
r_Hef = Hef[::-1]
ref_height=np.mean(r_Hef,axis=1)
ref_height=np.around(ref_height)
region_lat = np.around(region_lat,decimals=1)

In [ ]:
# Track plot
fig = plt.figure(figsize=(8,8)) 
m = Basemap(llcrnrlon=115, urcrnrlon=130., llcrnrlat=10., urcrnrlat=30.,resolution='l')
m.drawcoastlines(linewidth=1)
m.drawparallels(np.arange(-90., 120., 5.), labels=[1, 0, 0, 0])
m.drawmeridians(np.arange(-180., 181., 5.), labels=[0, 0, 0, 1])
m.scatter(mo_longitude[:], latitude[:] , c=MODIS_cloud_fraction[:] ,cmap=plt.cm.get_cmap('jet',10), vmin=10, vmax=120)
#plt.show()

In [ ]:
#雷達回波 plot
fig = plt.figure(figsize=(20,7)) 
plt.pcolormesh(r_Ref[20:100,:],cmap=plt.cm.get_cmap('jet',25), vmin=-25, vmax=25)
plt.colorbar(aspect=15, pad=0.02)
plt.title('Radar Reflectivity',fontsize=18)
#XY軸設定指令
plt.yticks(np.arange(0,80,step=8), (ref_height[20:100:8,]))
plt.xticks(np.arange(99,region_array_size[0],step=120), (region_lat[99:region_array_size[0]:120,0]))
plt.tick_params(axis="x", labelsize=20)
plt.tick_params(axis="y", labelsize=15)
plt.ylabel('Height(m)',color='k',fontsize=18)
plt.xlabel('Latitude',color='k',fontsize=18)
#plt.show()